In [27]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.datasets import load_iris
from sklearn.pipeline import Pipeline
import pickle

In [28]:
dataset = pd.read_csv('news.csv')
x = dataset['text']
y = dataset['label']


In [29]:
dataset.head()


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [30]:
dataset.shape


(6335, 4)

In [31]:
dataset.isnull().any()


Unnamed: 0    False
title         False
text          False
label         False
dtype: bool

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)


In [33]:
tfidf_vectorizer=TfidfVectorizer(stop_words= 'english' , max_df=0.7)
tfidf_train=tfidf_vectorizer.fit_transform(x_train)
tfidf_test=tfidf_vectorizer.transform(x_test)


In [34]:
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)
y_pred=pac.predict(tfidf_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')


Accuracy: 92.98%


In [35]:
pipeline = Pipeline([('tfidf', TfidfVectorizer(stop_words= 'english')),
                    ('nbmodel', MultinomialNB())])


In [36]:
pipeline.fit(x_train,y_train)


Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words='english')),
                ('nbmodel', MultinomialNB())])

In [37]:
score=pipeline.score(x_test,y_test)
print('accuracy',score)


accuracy 0.824782951854775


In [38]:
pred = pipeline.predict(x_test)


In [39]:
print(classification_report(y_test, pred))


              precision    recall  f1-score   support

        FAKE       0.97      0.67      0.79       633
        REAL       0.75      0.98      0.85       634

    accuracy                           0.82      1267
   macro avg       0.86      0.82      0.82      1267
weighted avg       0.86      0.82      0.82      1267



In [40]:
print(confusion_matrix(y_test, pred))


[[426 207]
 [ 15 619]]


In [41]:
with open('model.pickle', 'wb') as handle:
    pickle.dump(pipeline, handle, protocol=pickle.HIGHEST_PROTOCOL)
